In [15]:
# !pip install langchain==0.0.346
# !pip install newspaper3k==0.2.8
# !pip install pandas==2.1.3
# !pip install deltalake==0.14.0
# !pip install tabulate==0.9.0
# !pip install googlenews
# !pip install gnews==0.3.6

# !pip show gnews

In [16]:
import sys
sys.path.append('..')

# from langchain.document_loaders import NewsURLLoader
import nltk
# nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from GoogleNews import GoogleNews
from gnews import GNews

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
# pd.set_option('display.expand_frame_repr', False)

from datetime import datetime, timedelta
import pytz

from tabulate import tabulate

from deltalake import DeltaTable
from deltalake.writer import write_deltalake


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [17]:
def print_df(df):
    return tabulate(df, headers='keys', tablefmt='html')

def convert_url_to_parsed_article(url):
    try:
        article = google_news.get_full_article(url)
        article.nlp()
        return article
    except:
        return None

def convert_article_to_text(article):
    try:
        return article.text
    except:
        return None

def convert_article_to_html(article):
    try:
        return article.html
    except:
        return None

def convert_article_to_summary(article):
    try:
        return article.summary
    except:
        return None
    
def convert_article_to_keywords(article):
    try:
        return article.keywords
    except:
        return None
    


In [18]:
# Params
table_path = '../data/articles-ingest'
# --------------------------------------------
query = "Return to Office"

days_lookback = 30*6
max_results = 1000
current_datetime = datetime.now().astimezone(pytz.utc)
current_date = current_datetime.date()
start_date = (current_datetime - timedelta(days=days_lookback)).date()

# --------------------------------------------

start_date_str = str(start_date.strftime("%Y-%m-%d"))
current_date_str = str(current_date.strftime("%Y-%m-%d"))

print(
    f"""
    end_date: {current_date}
    start_date: {start_date}
    
    
    days_lookback: {days_lookback}
    max_results: {max_results}
    """
)


    end_date: 2023-12-07
    start_date: 2023-06-10
    
    
    days_lookback: 180
    max_results: 1000
    


In [19]:
exclude_websites = ['reuters.com','thestreet.com','digitalchew.com']

google_news = GNews(language='en', 
                    country='US', 
                    start_date=start_date, 
                    end_date=current_date, 
                    # period='7d', 
                    max_results=max_results, 
                    exclude_websites=exclude_websites,
                    # proxy=proxy
                    )

article_results = google_news.get_news(query)
# article_results

In [20]:
df = pd.DataFrame(data=article_results)
df['query'] = query
df['query end date'] = current_datetime
df['published datetime'] = df['published date'].apply(lambda x: datetime.strptime(x, '%a, %d %b %Y %H:%M:%S %Z'))
df['published date'] = df['published datetime'].dt.date
df['published date str'] = df['published date'].astype('datetime64[ns]').dt.strftime('%Y-%m-%d').astype(str)


In [21]:
df['article object'] = df['url'].apply(lambda x: convert_url_to_parsed_article(x))

An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/sites/chriswestfall/2023/10/23/amazon-empowers-managers-to-fire-employees-who-wont-return-to-office/ on URL https://news.google.com/rss/articles/CBMie2h0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvY2hyaXN3ZXN0ZmFsbC8yMDIzLzEwLzIzL2FtYXpvbi1lbXBvd2Vycy1tYW5hZ2Vycy10by1maXJlLWVtcGxveWVlcy13aG8td29udC1yZXR1cm4tdG8tb2ZmaWNlL9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en
An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/sites/shephyken/2023/09/24/nine-out-of-10-companies-will-require-employees-to-return-to-the-office/ on URL https://news.google.com/rss/articles/CBMiemh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvc2hlcGh5a2VuLzIwMjMvMDkvMjQvbmluZS1vdXQtb2YtMTAtY29tcGFuaWVzLXdpbGwtcmVxdWlyZS1lbXBsb3llZXMtdG8tcmV0dXJuLXRvLXRoZS1vZmZpY2Uv0gEA?oc=5&hl=en-US&gl=US&

In [ ]:
df['article text'] = df['article object'].apply(lambda x: convert_article_to_text(x) )
df['article html'] = df['article object'].apply(lambda x: convert_article_to_html(x) )
df['article summary'] = df['article object'].apply(lambda x: convert_article_to_summary(x) )
df['article keywords'] = df['article object'].apply(lambda x: convert_article_to_keywords(x) )
df = df.drop(columns=['article object'])

In [ ]:
write_deltalake(
    table_path,
    df,
    partition_by=['query', 'published date str'],
    mode='overwrite',
    partition_filters=[
        ('query', '=', query),
        ('published date str', '>=', start_date_str), 
        ('published date str', '<=', current_date_str),
    ]
)

In [ ]:
DeltaTable(table_path).to_pandas()

,title,description,published date,url,publisher,query,query end date,published datetime,published date str,article text,article html,article summary,article keywords
0,The Ozempic Craze Could Put These Companies on...,The Ozempic Craze Could Put These Companies on...,2023-08-18,https://news.google.com/rss/articles/CBMiY2h0d...,"{'href': 'https://www.wsj.com', 'title': 'The ...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-08-18 07:00:00,2023-08-18,None,None,None,None
1,Eli Lilly Just Got One Step Closer to Dethroni...,Eli Lilly Just Got One Step Closer to Dethroni...,2023-11-19,https://news.google.com/rss/articles/CBMiXWh0d...,"{'href': 'https://www.fool.com', 'title': 'The...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-11-19 08:00:00,2023-11-19,Just about every evening when I sit down for d...,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",Let's break down what Eli Lilly has in its pip...,"[step, eli, closer, drug, lilly, ozempic, buy,..."
2,Ozempic Sales Are Skyrocketing Faster Than Exp...,Ozempic Sales Are Skyrocketing Faster Than Exp...,2023-10-17,https://news.google.com/rss/articles/CBMiXWh0d...,"{'href': 'https://www.fool.com', 'title': 'The...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-10-17 07:00:00,2023-10-17,Everything seems to be going right for Novo No...,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",Ozempic sales are skyrocketing faster than exp...,"[expected, drug, skyrocketing, lilly, ozempic,..."
3,Will Ozempic-type obesity drugs pay off for so...,Will Ozempic-type obesity drugs pay off for so...,2023-10-17,https://news.google.com/rss/articles/CBMiUWh0d...,"{'href': 'https://www.statnews.com', 'title': ...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-10-17 07:00:00,2023-10-17,"Our expensive obesity drugs are worth it, Novo...","<!DOCTYPE html>\n<html lang=""en-US"">\n\t<head>...","Our expensive obesity drugs are worth it, Novo...","[studies, eli, lilly, wide, worth, novo, expen..."
4,"New Playground, New Court, New Classroom? Than...","New Playground, New Court, New Classroom? Than...",2023-10-17,https://news.google.com/rss/articles/CBMiVGh0d...,"{'href': 'https://www.wsj.com', 'title': 'The ...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-10-17 07:00:00,2023-10-17,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Newsroom unions' return to office negotiations...,Newsroom unions' return to office negotiations...,2023-08-28,https://news.google.com/rss/articles/CBMiY2h0d...,"{'href': 'https://digiday.com', 'title': 'Digi...",Return to Office,2023-12-07 03:12:20.091268,2023-08-28 07:00:00,2023-08-28,"With the summer season winding down, some medi...","<!DOCTYPE html>\n<html lang=""en-US"">\n<head>\n...","With the summer season winding down, some medi...","[employees, return, unions, heat, fall, approa..."
266,Future Of Work: Is A Return To The Office Inev...,Future Of Work: Is A Return To The Office Inev...,2023-11-06,https://news.google.com/rss/articles/CBMiZ2h0d...,"{'href': 'https://www.forbes.com', 'title': 'F...",Return to Office,2023-12-07 03:12:20.091268,2023-11-06 08:00:00,2023-11-06,None,None,None,None
267,Boeing is letting top executives work in small...,Boeing is letting top executives work in small...,2023-09-12,https://news.google.com/rss/articles/CBMiZmh0d...,"{'href': 'https://fortune.com', 'title': 'Fort...",Return to Office,2023-12-07 03:12:20.091268,2023-09-12 07:00:00,2023-09-12,"As the COVID pandemic gradually subsided, Boei...","<!DOCTYPE html><html lang=""en""><head><meta cha...","As the COVID pandemic gradually subsided, Boei...","[employees, return, wsj, team, work, office, b..."
268,1 in 10 Home Sellers Are Moving Due to Return-...,1 in 10 Home Sellers Are Moving Due to Return-...,2023-09-13,https://news.google.com/rss/articles/CBMiQGh0d...,"{'href': 'https://www.redfin.com', 'title': 'R...",Return to Office,2023-12-07 03:12:20.091268,2023-09-13 07:00:00,2023-09-13,"As a data journalist, Lily is passionate about